In [0]:
%pip install --upgrade --force-reinstall /dbfs/tmp/insurance-0.0.1-py3-none-any.whl

In [0]:
%restart_python

In [0]:
# Databricks notebook source
# COMMAND ----------
import os
import time
from typing import Dict, List

import requests
from pyspark.dbutils import DBUtils
from pyspark.sql import SparkSession
from loguru import logger
from insurance.config import ProjectConfig
from insurance.serving.fe_model_serving  import  FeatureLookupServing


# spark session

spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

# get environment variables
os.environ["DBR_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
os.environ["DBR_HOST"] = spark.conf.get("spark.databricks.workspaceUrl")

# Load project config
config = ProjectConfig.from_yaml(config_path="../project_config.yml")
catalog_name = config.catalog_name
schema_name = config.schema_name
endpoint_name = "insurance-model-serving-fe"

In [0]:
# Initialize Feature Lookup Serving Manager
feature_model_server = FeatureLookupServing(
    model_name=f"{catalog_name}.{schema_name}.insurance_model_fe_lightgbm",
    endpoint_name=endpoint_name,
    feature_table_name=f"{catalog_name}.{schema_name}.insurance_features",
)


In [0]:
# Create the online table for house features
feature_model_server.create_online_table()

In [0]:
# Deploy the model serving endpoint with feature lookup
feature_model_server.deploy_or_update_serving_endpoint()

In [0]:
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import monotonically_increasing_id, col
from pyspark.sql.types import DoubleType, LongType, IntegerType
from pyspark.sql.types import FloatType

train_set = spark.table(f"{config.catalog_name}.{config.schema_name}.train_set").toPandas()



# Create a sample request body
required_columns = ["Id", "sex", "smoker", "region", "age", "bmi", "children"]

sampled_records = train_set[required_columns].sample(n=1000, replace=True).to_dict(orient="records")

dataframe_records = [[record] for record in sampled_records]

logger.info(train_set.dtypes)
logger.info(dataframe_records[0])

In [0]:
import json
print(json.dumps(dataframe_records[0][0], indent=2))

In [0]:
# Call the endpoint with one sample record
def call_endpoint(record):
    """
    Calls the model serving endpoint with a given input record.
    """
    serving_endpoint = f"https://{os.environ['DBR_HOST']}/serving-endpoints/{endpoint_name}/invocations"

    response = requests.post(
        serving_endpoint,
        headers={"Authorization": f"Bearer {os.environ['DBR_TOKEN']}"},
        json={"dataframe_records": record},
    )
    return response.status_code, response.text

In [0]:
status_code, response_text = call_endpoint(dataframe_records[0])
print(f"Response Status: {status_code}")
print(f"Response Text: {response_text}")
